In [1]:
#test1
import tensorflow

In [69]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:' , x_train.shape)
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]

Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
#https://stats.stackexchange.com/questions/211436/why-normalize-images-by-subtracting-datasets-image-mean-instead-of-the-current
x_test = x_test - mean_image
"""In the ytrain it is preliminarily is in numbers just to indicate the particular class tf.keras.utils.to_categorical(y_train, num_classes=K) does is 
Converting the number to a binary vector
Ex: 
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)
tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)
"""
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))#shaping images into a vector Din=32x32x3
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
std=1e-5
w1 = std*np.random.randn(Din, K)#initialization
b1 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
batch_size = Ntr #for the batch size whole dataset is considered

iterations =3
lr =0.8
lr_decay=0.001#learning rate is decreased in each iteration
reg =0.5#regularization term which stops growth of weights
loss_history = []
train_acc_history = []
val_acc_history = []

seed = 0
rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    #indices = np.arange(Ntr)#generating image
    #rng.shuffle(indices)
    # Forward pass
    loss,cache=forward_prop(W=w1,b=b1,X=x_train,Y=y_train,reg=reg)
    print(loss)
    y_hat=cache["y_hat"]
    A=cache["A"]

    # Backward pass
    dw1,db1=back_prop(A=A,Y_hat=y_hat,W=w1,X=x_train,Y=y_train,reg=reg)
    w1=w1-lr*dw1
    b1=b1-lr*db1
# Printing accuracies and displaying w as images


x_train: (50000, 32, 32, 3)
w1: (3072, 10)
b1: (10,)
0.3124990546290997
0.31608315837848333
0.2669174327217398


In [21]:
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)
B=[[1,2,1,1],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
mse = np.mean((a - B)**2,axis=1)

print(mse)

def loss(W,b,X,Y,reg):
    print(W.shape,"W")
    print(b.shape,"b")
    print(X.shape,"X")
    print(Y.shape,"Y")
    prediction=np.dot(X,W)+np.transpose(b)
    print()
    m=X.shape[0]
    mse = np.sum(np.square((Y - prediction)**2))
    Loss=mse+ 1/(2*m)*reg*np.sum(W**2)
    return Loss

tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)
[1.5 0.  0.  0. ]


In [12]:
def sigmoid(z):
    z=np.array(z)
    return 1/(1+np.exp(-z))

sigmoid(B)

array([[0.73105858, 0.88079708, 0.73105858, 0.73105858],
       [0.5       , 0.73105858, 0.5       , 0.5       ],
       [0.5       , 0.5       , 0.73105858, 0.5       ],
       [0.5       , 0.5       , 0.5       , 0.73105858]])

In [68]:
def forward_prop(X,Y,b,W,reg):
    #print(W.shape,"W")
    #print(b.shape,"b")
    #print(X.shape,"X")
    #print(Y.shape,"Y")
    A=np.dot(X,W)+np.transpose(b)
    #print(A.shape,"A")
    y_hat=sigmoid(A)
    m=X.shape[0]
    #print(m)
    mse =1/(2*m)* np.sum(np.square((Y - y_hat)**2))
    Loss=mse+ 1/(2*m)*reg*np.sum(W**2)
    cache={"A":A,"y_hat":y_hat}
    return Loss,cache

In [49]:
loss,cache=forward_prop(W=w1,b=b1,X=x_train,Y=y_train,reg=reg)
print(loss)
y_hat=cache["y_hat"]
A=cache["A"]

(3072, 10) W
(10,) b
(50000, 3072) X
(50000, 10) Y
(50000, 10) A
50000
0.31249970513070713


In [65]:

def back_prop(A,W,X,Y,Y_hat,reg):
    m=Y.shape[0]
    dy_hat =Y_hat-Y
    #print(dy_hat.shape,"dy_hat")
    dA = dy_hat *sigmoid(A)*(1-sigmoid(A))
    #print(dA.shape,"dA")
    dW =(1/float(m))*(np.dot(X.T, dA)+reg*W)
    db=(1/float(m))*np.sum(dA,axis=0)
    #print(dW.shape)
    #print(db.shape)
    return dW,db

In [66]:
back_prop(A=A,Y_hat=y_hat,W=w1,X=x_train,Y=y_train,reg=reg)

(array([[-1.40129866e-03, -5.90441550e-04,  5.67232783e-04, ...,
         -4.19029003e-04, -1.52425744e-03, -3.59148941e-03],
        [-2.46889924e-03, -2.14337224e-04,  5.00707325e-04, ...,
         -5.29186497e-04, -2.62208550e-03, -4.04770045e-03],
        [-4.47781285e-03, -3.38515906e-04,  1.68333211e-03, ...,
         -4.82374567e-04, -4.54256479e-03, -5.10905912e-03],
        ...,
        [-3.15761586e-04, -6.91707276e-04,  1.48683954e-04, ...,
         -1.33483174e-03,  2.96574120e-03, -1.26495232e-03],
        [-9.94508399e-04, -5.53026108e-04, -8.57725095e-06, ...,
         -8.92934096e-04,  1.57972060e-03, -9.78336180e-04],
        [-2.28748909e-03, -1.20831754e-03,  7.39150279e-04, ...,
          5.82655407e-04, -3.43120827e-04, -1.38096240e-03]]),
 array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]))